<a href="https://colab.research.google.com/github/jiw3026/SML/blob/main/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train=pd.read_csv('/content/drive/MyDrive/dacon/train.csv')
test=pd.read_csv('/content/drive/MyDrive/dacon/test.csv')

In [4]:
train.isnull().sum()

id                          0
hour                        0
hour_bef_temperature        2
hour_bef_precipitation      2
hour_bef_windspeed          9
hour_bef_humidity           2
hour_bef_visibility         2
hour_bef_ozone             76
hour_bef_pm10              90
hour_bef_pm2.5            117
count                       0
dtype: int64

In [5]:
test.isnull().sum()

id                         0
hour                       0
hour_bef_temperature       1
hour_bef_precipitation     1
hour_bef_windspeed         1
hour_bef_humidity          1
hour_bef_visibility        1
hour_bef_ozone            35
hour_bef_pm10             37
hour_bef_pm2.5            36
dtype: int64

In [6]:
train.fillna(0,inplace=True)
test.fillna(0,inplace=True)

In [7]:
colL=['hour','hour_bef_temperature','hour_bef_precipitation','hour_bef_windspeed','hour_bef_humidity','hour_bef_visibility','hour_bef_visibility','hour_bef_ozone','hour_bef_pm10','hour_bef_pm2.5','count']

train_iqr=train.copy()
for col in colL:
  Q1=train_iqr[col].quantile(.25)
  Q3=train_iqr[col].quantile(.75)
  IQR =Q3-Q1
  train_del=train_iqr[((Q1-1.5*IQR)>train_iqr[col])|((Q3+1.5*IQR)<train_iqr[col])]
  train_iqr=train_iqr.drop(train_del.index,axis=0)
train_iqr

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
1,6,13,20.1,0.0,1.4,48.0,916.0,0.042,73.0,40.0,159.0
2,7,6,13.9,0.0,0.7,79.0,1382.0,0.033,32.0,19.0,26.0
5,13,2,13.6,0.0,1.7,80.0,1073.0,0.027,34.0,15.0,39.0
6,14,3,10.6,0.0,1.5,58.0,1548.0,0.038,62.0,33.0,23.0
7,16,21,16.0,0.0,6.0,21.0,1961.0,0.050,90.0,28.0,146.0
...,...,...,...,...,...,...,...,...,...,...,...
1454,2174,4,16.8,0.0,1.6,53.0,2000.0,0.031,37.0,27.0,21.0
1455,2175,3,10.8,0.0,3.8,45.0,2000.0,0.039,34.0,19.0,20.0
1456,2176,5,18.3,0.0,1.9,54.0,2000.0,0.009,30.0,21.0,22.0
1457,2178,21,20.7,0.0,3.7,37.0,1395.0,0.082,71.0,36.0,216.0


In [8]:
train_x=train_iqr.drop(['count'],axis=1)
train_y=train_iqr['count']
X_train,X_test,y_train,y_test=train_test_split(train_x,train_y,test_size=0.25,random_state=42)

In [9]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
scaler.fit(X_train)
X_train=scaler.transform(X_train)

In [10]:
X_test=scaler.transform(X_test)

In [11]:
#train데이터정확도
from sklearn.svm import SVR
model=SVR(kernel='poly')
model.fit(X_train,y_train)
pred_train=model.predict(X_train)
model.score(X_train,y_train)

0.4227418049315229

In [12]:
#test데이터정확도
pred_test=model.predict(X_test)
model.score(X_test,y_test)

0.4205565595047416

In [13]:
#RMSE
from sklearn.metrics import mean_squared_error
MSE_train=mean_squared_error(y_train,pred_train)
MSE_test=mean_squared_error(y_test,pred_test)
print('train데이터 :',np.sqrt(MSE_train))
print('test데이터 :',np.sqrt(MSE_test))

train데이터 : 59.137950688476245
test데이터 : 63.729168341972944


In [14]:
#하이퍼파라미터 조정 : Grid Search
from sklearn.model_selection import GridSearchCV
param_grid={'C':[0.01,0.1,1,10],'kernel':['rbf'],'gamma':[0.01,0.1,1,10]}
grid_search=GridSearchCV(SVR(),param_grid,cv=5,scoring='neg_mean_squared_error')
grid_search.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [0.01, 0.1, 1, 10], 'gamma': [0.01, 0.1, 1, 10],
                         'kernel': ['rbf']},
             scoring='neg_mean_squared_error')

In [15]:
#최적의 하이퍼파라미터
print('Best Parameter : {}'.format(grid_search.best_params_))
#train데이터 정확도
print('Best Train Score : {:.4f}'.format(grid_search.best_score_))
#test데이터 정확도
print('Best Test Score : {:.4f}'.format(grid_search.score(X_test,y_test)))

Best Parameter : {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
Best Train Score : -2637.3877
Best Test Score : -2426.0385


In [16]:
scores_df=pd.DataFrame(grid_search.cv_results_)
scores_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.038421,0.002044,0.010994,0.001828,0.01,0.01,rbf,"{'C': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}",-5226.251562,-6531.079939,-6137.122363,-6942.495833,-5877.609009,-6142.911741,583.214233,10
1,0.038411,0.002033,0.009940,0.000338,0.01,0.1,rbf,"{'C': 0.01, 'gamma': 0.1, 'kernel': 'rbf'}",-5200.352572,-6491.827837,-6106.816063,-6908.706420,-5846.787451,-6110.898069,579.736118,9
2,0.039177,0.006486,0.010689,0.000917,0.01,1,rbf,"{'C': 0.01, 'gamma': 1, 'kernel': 'rbf'}",-5247.948809,-6556.725544,-6160.056510,-6970.599023,-5901.783029,-6167.422583,585.269142,13
3,0.051639,0.003105,0.009986,0.000157,0.01,10,rbf,"{'C': 0.01, 'gamma': 10, 'kernel': 'rbf'}",-5250.065362,-6559.476267,-6162.254434,-6973.093725,-5904.193284,-6169.816614,585.430452,16
4,0.038850,0.000806,0.010254,0.000278,0.1,0.01,rbf,"{'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}",-5036.376142,-6265.984181,-5918.256196,-6687.635811,-5662.532241,-5914.156914,557.832035,7
5,0.040260,0.002245,0.010272,0.000345,0.1,0.1,rbf,"{'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf'}",-4811.883355,-5943.068641,-5669.441908,-6400.155900,-5418.319426,-5648.573846,530.114387,6
6,0.040736,0.004341,0.010340,0.000413,0.1,1,rbf,"{'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}",-5229.970048,-6530.445722,-6140.119826,-6948.075817,-5879.398227,-6145.601928,583.325859,11
7,0.049388,0.001236,0.010850,0.001570,0.1,10,rbf,"{'C': 0.1, 'gamma': 10, 'kernel': 'rbf'}",-5250.488423,-6557.892743,-6162.054706,-6972.972977,-5903.446849,-6169.371139,585.122151,15
8,0.036862,0.000538,0.009578,0.000058,1,0.01,rbf,"{'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}",-3797.902267,-4721.445393,-4500.401619,-5033.558719,-4300.927790,-4470.847158,415.221546,4
9,0.037083,0.000167,0.010169,0.000665,1,0.1,rbf,"{'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}",-3282.590054,-3867.308369,-3840.252007,-4297.062835,-3696.378673,-3796.718388,326.073209,3


In [18]:
scores_df['rmse']=np.sqrt(-scores_df['mean_test_score'])

In [21]:
rmse_df=scores_df.loc[:,['param_C','param_gamma','param_kernel','mean_test_score','rmse']].sort_values('mean_test_score',ascending=False)
rmse_df

,param_C,param_gamma,param_kernel,mean_test_score,rmse
13,10,0.1,rbf,-2637.387717,51.355503
12,10,0.01,rbf,-2892.267377,53.779805
9,1,0.1,rbf,-3796.718388,61.617517
8,1,0.01,rbf,-4470.847158,66.864394
14,10,1,rbf,-4643.720871,68.144852
5,0.1,0.1,rbf,-5648.573846,75.156995
4,0.1,0.01,rbf,-5914.156914,76.903556
10,1,1,rbf,-5935.688814,77.043422
1,0.01,0.1,rbf,-6110.898069,78.172233
0,0.01,0.01,rbf,-6142.911741,78.376730
